In [ ]:
using JuMP
import Ipopt
using Distributions
using Dates
using Combinatorics
using JLD2
include("utilities.jl")
folder = pwd()  

In [ ]:
actions_states = [2 3; 3 4; 3 5]
D = [[[3], [2 1], [1 1 1]], [[4], [3 1], [2 2], [2 1 1], [1 1 1 1]], [[5], [3 2], [4 1], [3,1,1], [2,2,1], [2,1,1,1], [1,1,1,1,1]]]
iterations = 1

In [ ]:
for i in 1:size(actions_states)[1]
    (nA, nS) =  actions_states[i,:]
    for j in 1:length(D[i])
        dₒ = D[i][j]
        nO = length(dₒ)
        println("nA = ", nA, ", nS = ", nS, ", dₒ = ", dₒ)
        for it in 1:iterations
            # Data
            γ = 0.9
            Mβ = load_object(string(folder,"/Data/beta_", nA, "_", nS, "_", nO, "_", j, "_", it))
            Mα = load_object(string(folder,"/Data/alpha_", nA, "_", nS, "_", nO, "_", j, "_", it))
            r = load_object(string(folder,"/Data/r_", nA, "_", nS, "_", nO, "_", j, "_", it))
            μ = load_object(string(folder,"/Data/mu_", nA, "_", nS, "_", nO, "_", j, "_", it))

            # Info
            N_nonSing = []; N_nonSing_real = []; N_nonSing_pos = [];
            N_sing = []; N_sing_real = []; N_sing_pos = [];
            R_nonSing_max = -Inf; R_sing_max = -Inf
            η_nonSing = []; η_sing = []; MAXη = ""

            ### Solve Systems
            @var η[1:nS, 1:nA];
            lEqs = linearEqualitiesPOMDP(η, γ, μ, Mα)
            pEqs = polynomialEqualitiesPOMDP(η, Mβ, nA)
            h = Vector{Expression}(vcat(lEqs, pEqs));
            f = ObjectiveFunction(η, r);

            vdₒ = Array{Int64}(sum(eachcol(Mβ)))
            Aₒ = []; countAₒ = []
            for d in vdₒ
                subsets = collect(Combinatorics.powerset(1:nA, maximum([0,nA-d]), nA-1))
                push!(Aₒ, subsets)
                push!(countAₒ, Vector(1:length(subsets)))
            end

            indices = collect(Base.product(countAₒ...))

            FACES = []
            SOLS = []
            tLagrange=@elapsed begin
                for idx in indices
                    face = [Aₒ[x][idx[x]] for x in 1:length(Aₒ)]
                    f₀ = f; h₀ = h;
                    faceVars = []
                    for o in 1:nO
                        states = findall(x->x==1, Mβ[o, :])
                        for a in 1:length(face[o])
                            zeroVars = η[states,face[o][a]]
                            faceVars = vcat(faceVars, zeroVars)
                            f₀ = subs(f₀, variables(zeroVars) => zeros(length(zeroVars)))
                            h₀ = subs(h₀, variables(zeroVars) => zeros(length(zeroVars)))
                        end
                    end
                    boundV = variables(Vector{Variable}(faceVars))
                    intV = setdiff(reshape(η, nS*nA), faceVars);
                    ηs = reshape(η, nA*nS)
                    ηs = subs(ηs, variables(boundV) => zeros(length(boundV)))
                    FACES = append!(FACES, [ηs])

                    S = lagrangeSystem(f₀, uniqueEqs(h₀))
                    rs = HomotopyContinuation.solve(S, show_progress=false)

                    sols = solutions(rs, only_nonsingular=false)
                    if length(sols) > 0

                        sols_nonSing = solutions(rs, only_nonsingular=true)
                        sols_sing = setdiff(sols, sols_nonSing)

                        sols_nonSing = [sols_nonSing[ii][1:length(intV)] for ii in 1:length(sols_nonSing)]
                        ηs_nonSing = [subs(ηs, variables(Vector{Variable}(intV)) => sols_nonSing[ii]) for ii in 1:length(sols_nonSing)]

                        sols_sing = [sols_sing[ii][1:length(intV)] for ii in 1:length(sols_sing)]
                        ηs_sing = [subs(ηs, variables(Vector{Variable}(intV)) => sols_sing[ii]) for ii in 1:length(sols_sing)]

                        (n_nonSing, n_nonSing_real, n_nonSing_pos, n_sing, n_sing_real, n_sing_pos,
                        η_nonSing_max, r_nonSing_max, η_sing_max, r_sing_max, maxη) = analyzeSolutions_η(ηs_nonSing, ηs_sing, reshape(r, nS*nA))

                        # Info
                        N_nonSing = append!(N_nonSing, n_nonSing);
                        N_nonSing_real = append!(N_nonSing_real, n_nonSing_real);
                        N_nonSing_pos = append!(N_nonSing_pos, n_nonSing_pos);
                        N_sing = append!(N_sing, n_sing);
                        N_sing_real = append!(N_sing_real, n_sing_real);
                        N_sing_pos = append!(N_sing_pos, n_sing_pos);
                        SOLS = append!(SOLS, length(unique(sols)))

                        if r_nonSing_max > R_nonSing_max
                            R_nonSing_max = r_nonSing_max
                            η_nonSing = η_nonSing_max
                            MAXη = maxη
                        end

                        if r_sing_max > R_sing_max
                            R_sing_max = r_sing_max
                            η_sing = η_sing_max
                            MAXη = maxη
                        end
                    else
                        N_nonSing = append!(N_nonSing, 0);
                        N_nonSing_real = append!(N_nonSing_real, 0);
                        N_nonSing_pos = append!(N_nonSing_pos, 0);
                        N_sing = append!(N_sing, 0);
                        N_sing_real = append!(N_sing_real, 0);
                        N_sing_pos = append!(N_sing_pos, 0);
                        SOLS = append!(SOLS, 0)
                    end
                end
            end

            infoFile = string(folder, "/Results/Lagrange/Lagrange_relevant_info_", nA, "_", nS, "_", nO, "_", j, "_", it, ".txt")
            open(infoFile, "w") do f
                println(f, ("Lagrange relevant", tLagrange, R_nonSing_max, R_sing_max, MAXη, sum(SOLS),
                 sum(N_nonSing), sum(N_nonSing_real), sum(N_nonSing_pos), sum(N_sing), sum(N_sing_real), sum(N_sing_pos)))
            end
        end
    end
end